In [ ]:
# 트렌스포머 2차 시도 (시계열 분석하기)

In [12]:
import math
import random

def generate_data(num_trials):
    trials = []
    for _ in range(num_trials):
        count = 0
        while True:
            count += 1
            if random.randint(1, 125) == 1:
                break
        trials.append(count)
    return trials

x_data = generate_data(3200)

def apply_formula(x):
    # 여기에 당신의 공식을 적용합니다.
    # 예: y = x * 2
    y = round(1-math.exp(-x/125),4)
    return y

def generate_y_list(x_list):
    y_list = []
    for x in x_list:
        y = apply_formula(x)
        y_list.append(y)
    return y_list

# Y 데이터 리스트 생성
y_data = generate_y_list(x_data)

print("X 데이터:", x_data)
print("Y 데이터:", y_data)

X 데이터: [91, 42, 129, 107, 39, 506, 14, 86, 220, 45, 84, 648, 220, 10, 133, 189, 135, 3, 130, 294, 107, 108, 186, 346, 69, 327, 229, 4, 191, 765, 26, 8, 82, 120, 28, 24, 55, 61, 136, 148, 86, 15, 154, 93, 54, 114, 63, 185, 132, 19, 152, 45, 147, 1, 64, 448, 2, 157, 370, 108, 23, 146, 60, 130, 146, 152, 75, 23, 72, 52, 50, 535, 105, 144, 289, 45, 68, 77, 82, 148, 36, 414, 114, 258, 5, 43, 68, 222, 44, 28, 135, 198, 62, 27, 86, 119, 23, 289, 3, 210, 24, 112, 78, 111, 32, 275, 169, 473, 40, 70, 90, 41, 40, 410, 52, 213, 4, 78, 47, 194, 9, 58, 30, 5, 209, 303, 411, 138, 362, 130, 82, 111, 142, 56, 4, 104, 172, 14, 85, 203, 523, 45, 285, 172, 440, 87, 115, 56, 240, 140, 7, 119, 351, 102, 387, 7, 4, 59, 349, 18, 140, 20, 122, 136, 57, 336, 535, 390, 100, 145, 15, 18, 137, 79, 108, 76, 17, 66, 32, 20, 154, 193, 158, 31, 231, 247, 9, 377, 320, 323, 193, 20, 21, 9, 48, 38, 83, 80, 135, 60, 172, 136, 136, 151, 88, 265, 125, 29, 236, 47, 91, 77, 71, 297, 209, 103, 221, 45, 50, 376, 48, 19, 284, 26

In [15]:
# 주어진 1차원 리스트
#original_list = [1, 2, 3, ..., n]  # 여기서 n은 리스트의 마지막 요소

# 10개씩 묶어 2차원 리스트로 변환
data_x = [x_data[i:i+8] for i in range(0, len(x_data), 8)]

# 결과 출력
print(data_x)

[[91, 42, 129, 107, 39, 506, 14, 86], [220, 45, 84, 648, 220, 10, 133, 189], [135, 3, 130, 294, 107, 108, 186, 346], [69, 327, 229, 4, 191, 765, 26, 8], [82, 120, 28, 24, 55, 61, 136, 148], [86, 15, 154, 93, 54, 114, 63, 185], [132, 19, 152, 45, 147, 1, 64, 448], [2, 157, 370, 108, 23, 146, 60, 130], [146, 152, 75, 23, 72, 52, 50, 535], [105, 144, 289, 45, 68, 77, 82, 148], [36, 414, 114, 258, 5, 43, 68, 222], [44, 28, 135, 198, 62, 27, 86, 119], [23, 289, 3, 210, 24, 112, 78, 111], [32, 275, 169, 473, 40, 70, 90, 41], [40, 410, 52, 213, 4, 78, 47, 194], [9, 58, 30, 5, 209, 303, 411, 138], [362, 130, 82, 111, 142, 56, 4, 104], [172, 14, 85, 203, 523, 45, 285, 172], [440, 87, 115, 56, 240, 140, 7, 119], [351, 102, 387, 7, 4, 59, 349, 18], [140, 20, 122, 136, 57, 336, 535, 390], [100, 145, 15, 18, 137, 79, 108, 76], [17, 66, 32, 20, 154, 193, 158, 31], [231, 247, 9, 377, 320, 323, 193, 20], [21, 9, 48, 38, 83, 80, 135, 60], [172, 136, 136, 151, 88, 265, 125, 29], [236, 47, 91, 77, 71, 29

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

data = [torch.tensor(sequence, dtype=torch.float32) for sequence in data_x]
dataset = TensorDataset(torch.stack(data))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout):
        super(TransformerModel, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dim_feedforward=hidden_size, dropout=dropout),
            num_layers=num_layers
        )

    def forward(self, src):
        output = self.transformer_encoder(src)
        return output

# 모델 초기화
input_size = len(data_x[0])  # 시퀀스의 길이를 입력 크기로 사용
model = TransformerModel(input_size=input_size, hidden_size=512, num_layers=4, num_heads=8, dropout=0.1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정
num_epochs = 10  # 에포크 수를 설정
for epoch in range(num_epochs):
    for inputs, _ in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)  # 예시는 자기회귀적 예측을 가정
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 예측
# 예시: model(torch.tensor(your_new_sequence, dtype=torch.float32))


ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
import math
import random

def generate_data(num_trials):
    trials = []
    for _ in range(num_trials):
        count = 0
        while True:
            count += 1
            if random.randint(1, 125) == 1:
                break
        trials.append(count)
    return trials

x_data = generate_data(3200)

data_x = [x_data[i:i+8] for i in range(0, len(x_data), 8)]

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 데이터를 2차원 텐서로 변환
data = [torch.tensor(sequence, dtype=torch.float32).unsqueeze(0) for sequence in data_x]
dataset = TensorDataset(torch.stack(data))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout):
        super(TransformerModel, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dim_feedforward=hidden_size, dropout=dropout),
            num_layers=num_layers
        )

    def forward(self, src):
        output = self.transformer_encoder(src)
        return output

input_size = len(data_x[0])  # 8
model = TransformerModel(input_size=input_size, hidden_size=512, num_layers=4, num_heads=1, dropout=0.1)  # num_heads를 1로 변경
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for inputs in dataloader:
        inputs = inputs[0]  # inputs는 튜플 형태이므로, 첫 번째 요소만 사용
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs) 
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 28335.3515625
Epoch 2, Loss: 34679.53125
Epoch 3, Loss: 30789.861328125
Epoch 4, Loss: 34923.25390625
Epoch 5, Loss: 30127.78515625
Epoch 6, Loss: 27533.744140625
Epoch 7, Loss: 36628.98046875
Epoch 8, Loss: 22560.15625
Epoch 9, Loss: 21032.380859375
Epoch 10, Loss: 29098.52734375
